# MLPerf Inference v2.0 - Reproducing Orin results with JetPack 4.6

# Table of Contents

1. [System](#system)
1. [Installation](#installation)
    1. [Clone the repo](#installation_repo)
    1. [Install the dependencies](#installation_deps)
    1. [Link the datasets](#installation_datasets)
    1. [Download the models](#installation_models)
    1. [Build the harness](#installation_harness)
1. [ResNet50](#resnet50)
    1. [Offline](#resnet50_offline)
        1. [Build](#resnet50_offline_build)
        1. [Performance](#resnet50_offline_performance)
        1. [Accuracy](#resnet50_offline_accuracy)
    1. [SingleStream](#resnet50_singlestream)
        1. [Build](#resnet50_singlestream_build)
        1. [Performance](#resnet50_singlestream_performance)
        1. [Accuracy](#resnet50_singlestream_accuracy)
    1. [MultiStream](#resnet50_multistream)
        1. [Build](#resnet50_multistream_build)
        1. [Performance](#resnet50_multistream_performance)
        1. [Accuracy](#resnet50_multistream_accuracy)
1. [SSD-ResNet34](#ssd-resnet34)
    1. [Offline](#ssd-resnet34_offline)
        1. [Build](#ssd-resnet34_offline_build)
        1. [Performance](#ssd-resnet34_offline_performance)
        1. [Accuracy](#ssd-resnet34_offline_accuracy)
    1. [SingleStream](#ssd-resnet34_singlestream)
        1. [Build](#ssd-resnet34_singlestream_build)
        1. [Performance](#ssd-resnet34_singlestream_performance)
        1. [Accuracy](#ssd-resnet34_singlestream_accuracy)
    1. [MultiStream](#ssd-resnet34_multistream)
        1. [Build](#ssd-resnet34_multistream_build)
        1. [Performance](#ssd-resnet34_multistream_performance)
        1. [Accuracy](#ssd-resnet34_multistream_accuracy)

<a name="system"></a>
# System: [NVIDIA Orin](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/systems/Orin_TRT.json)

## Stats: JetPack

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo -H python3 -m pip install jetson-stats -U
...
Successfully installed jetson-stats-3.1.0
</pre>

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; jetson_release

 - NVIDIA Jetson UNKNOWN
   * Jetpack UNKNOWN [L4T 34.1.0]
   * NV Power Mode: MAXN - Type: 0
   * jetson_stats.service: active
 - Libraries:
   * CUDA: NOT_INSTALLED
   * cuDNN: 8.3.2.49
   * TensorRT: 8.4.0.9
   * Visionworks: NOT_INSTALLED
   * OpenCV: 4.5.4 compiled CUDA: NO
   * VPI: ii libnvvpi2 2.0.9 arm64 NVIDIA Vision Programming Interface library
   * Vulkan: 1.3.203
</pre>

## OS: Ubuntu 18.04.5 LTS

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; cat /etc/lsb-release
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION=&quot;Ubuntu 20.04.4 LTS&quot;
</pre>

## CPU: 12-core ARMv81 @ 2200 MHz

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; lscpu
Architecture:                    aarch64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
CPU(s):                          12
On-line CPU(s) list:             0-11
Thread(s) per core:              1
Core(s) per socket:              4
Socket(s):                       3
Vendor ID:                       ARM
Model:                           1
Model name:                      ARMv8 Processor rev 1 (v8l)
Stepping:                        r0p1
CPU max MHz:                     2201.6001
CPU min MHz:                     115.2000
BogoMIPS:                        62.50
L1d cache:                       768 KiB
L1i cache:                       768 KiB
L2 cache:                        3 MiB
L3 cache:                        6 MiB
Vulnerability Itlb multihit:     Not affected
Vulnerability L1tf:              Not affected
Vulnerability Mds:               Not affected
Vulnerability Meltdown:          Not affected
Vulnerability Spec store bypass: Mitigation; Speculative Store Bypass disabled via prctl
Vulnerability Spectre v1:        Mitigation; __user pointer sanitization
Vulnerability Spectre v2:        Not affected
Vulnerability Srbds:             Not affected
Vulnerability Tsx async abort:   Not affected
Flags:                           fp asimd evtstrm aes pmull sha1 sha2 crc32 atomics fphp asimdhp cpuid asimdrdm lrcpc dcpop asimddp uscat ilrcpc flagm
</pre>

## GPU

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo jetson_clocks
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo jetson_clocks --show
SOC family:tegra234  Machine:NVIDIA Orin Jetson-Small Developer Kit
Online CPUs: 0-11
cpu0: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu1: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu10: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu11: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu2: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu3: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu4: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu5: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu6: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu7: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu8: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu9: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
GPU MinFreq=1300500000 MaxFreq=1300500000 CurrentFreq=1300500000
EMC MinFreq=204000000 MaxFreq=3199000000 CurrentFreq=3199000000 FreqOverride=1
DLA0_CORE MinFreq=0 MaxFreq=1536000000 CurrentFreq=1536000000
DLA0_FALCON MinFreq=0 MaxFreq=832000000 CurrentFreq=832000000
DLA1_CORE MinFreq=0 MaxFreq=1536000000 CurrentFreq=1536000000
DLA1_FALCON MinFreq=0 MaxFreq=832000000 CurrentFreq=832000000
PVA0_VPS0 MinFreq=0 MaxFreq=1152000000 CurrentFreq=1152000000
PVA0_AXI MinFreq=0 MaxFreq=832000000 CurrentFreq=832000000
FAN Dynamic Speed control=active hwmon3_pwm=88
NV Power Mode: MAXN
</pre>


<pre>
<font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; ck compile program:tool-print-cuda-devices
<font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; ck run program:tool-print-cuda-devices
...
GPU Device ID: 0
GPU Name: Orin
GPU compute capability: 8.7
CUDA driver version: 11.4
CUDA runtime version: 11.4
Global memory: 32110174208
Max clock rate: 1300.000000 MHz
Total amount of shared memory per block: 49152
Total number of registers available per block: 65536
Warp size: 32
Maximum number of threads per multiprocessor:  1536
Maximum number of threads per block: 1024
Max dimension size of a thread block X: 1024
Max dimension size of a thread block Y: 1024
Max dimension size of a thread block Z: 64
Max dimension size of a grid size X: 2147483647
Max dimension size of a grid size Y: 65535
Max dimension size of a grid size Z: 65535
</pre>


## Disks

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; df -h
Filesystem      Size  Used Avail Use% Mounted on
/dev/mmcblk0p1   57G   18G   37G  33% /
none             15G     0   15G   0% /dev
tmpfs            15G   52K   15G   1% /dev/shm
tmpfs           3.0G   11M  3.0G   1% /run
tmpfs           5.0M  4.0K  5.0M   1% /run/lock
tmpfs            15G     0   15G   0% /sys/fs/cgroup
/dev/mmcblk1p1  469G   36G  409G   9% /sd
tmpfs           3.0G   16K  3.0G   1% /run/user/124
tmpfs           3.0G  4.0K  3.0G   1% /run/user/1000
tmpfs           3.0G  8.0K  3.0G   1% /run/user/1001
</pre>

<a name="installation"></a>
# Installation

<a name="installation_repo"></a>
## Clone the v2.0 results repo

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; git clone https://github.com/mlcommons/inference_results_v2.0.git</pre>

<a name="installation_deps"></a>
## Install the dependencies

To [quote](https://github.com/mlcommons/inference_results_v2.0/tree/master/closed/Azure) Azure:
> Note that this might take a while, on the order of several hours.

<pre><font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; chmod +x inference_results_v2.0/closed/Azure/scripts/install_orin_jetson_dependencies.sh
<font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; time sudo inference_results_v2.0/closed/Azure/scripts/install_orin_jetson_dependencies.sh
<font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; python3 -m pip install onnx
</pre>

<a name="installation_datasets"></a>

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; echo 'export MLPERF_SCRATCH_PATH=/datasets/nvidia_scratch' >> $HOME/.bashrc
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; source $HOME/.bashrc
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; mkdir &dollar;MLPERF_SCRATCH_PATH
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; cd &dollar;MLPERF_SCRATCH_PATH
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/nvidia_scratch</b></font>&dollar; mkdir data models preprocessed_data
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/mlperf_scratch_path</b></font>&dollar; ls -la /datasets/nvidia_scratch/
total 20
drwxrwsr-x  3 anton dvdt 4096 Jan 28 11:10 <font color="#268BD2"><b>.</b></font>
drwxrwsr-x 17 root  dvdt 4096 Jan 28 09:53 <font color="#268BD2"><b>..</b></font>
lrwxrwxrwx  1 anton dvdt   64 Jan 28 11:10 <font color="#2AA198"><b>data</b></font> 
drwxrwsr-x  5 anton dvdt 4096 Jan 28 09:54 <font color="#268BD2"><b>models</b></font>
lrwxrwxrwx  1 anton dvdt   77 Jan 28 11:10 <font color="#2AA198"><b>preprocessed_data</b></font> 
</pre>

<a name="download_datasets"></a>
## Download the datasets
<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>:/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; bash code/bert/tensorrt/download_data.sh

<a name="installation_models"></a>
## Download the models

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>:/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; make download_model BENCHMARKS="resnet50,bert"
...
Finished downloading all the models!
</pre>

## Preprocess the dataset

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>:/datasets/inference_results_v2.0/closed/Azure</b></font>&dollar; make preprocess_data BENCHMARKS="resnet50,bert"
</pre>

## Build the binaries

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; git diff code/harness/CMakeLists.txt

diff --git a/closed/NVIDIA/code/harness/CMakeLists.txt b/closed/NVIDIA/code/harness/CMakeLists.txt
index 074c0bc25..b205e7691 100644
--- a/closed/NVIDIA/code/harness/CMakeLists.txt
+++ b/closed/NVIDIA/code/harness/CMakeLists.txt
@@ -503,48 +503,48 @@ else()
     )

     ######### RNN-T HARNESS ########
-    execute_process(COMMAND echo "Building RNN-T harness...")
+    #    execute_process(COMMAND echo "Building RNN-T harness...")

-    get_dali_paths(DALI_INCLUDE_DIR DALI_LIB_DIR DALI_LIBRARIES)
+    #    get_dali_paths(DALI_INCLUDE_DIR DALI_LIB_DIR DALI_LIBRARIES)

-    message(STATUS "DALI libraries DIR: " ${DALI_LIB_DIR})
-    message(STATUS "DALI include DIR: " ${DALI_INCLUDE_DIR})
+    #    message(STATUS "DALI libraries DIR: " ${DALI_LIB_DIR})
+    #    message(STATUS "DALI include DIR: " ${DALI_INCLUDE_DIR})

-    message(STATUS "DALI linked libraries: " ${DALI_LIBRARIES})
+    #    message(STATUS "DALI linked libraries: " ${DALI_LIBRARIES})

-    add_executable(harness_rnnt
-        harness_rnnt/main_rnnt.cc
-        harness_rnnt/rnnt_kernels.cu
-        common/logger.cpp
-    )
+    #    add_executable(harness_rnnt
+    #        harness_rnnt/main_rnnt.cc
+    #        harness_rnnt/rnnt_kernels.cu
+    #        common/logger.cpp
+    #    )

     # Add gencode compilation option for SOC (Xavier, Orin)
-    if (${IS_SOC})
-        message(STATUS "Compiling RNNT harness for SM${SOC_SM}.")
-        target_compile_options(harness_rnnt PRIVATE $<$<COMPILE_LANGUAGE:CUDA>: -gencode arch=compute_${SOC_SM},code=sm_${SOC_SM}>)
-    endif()
-
-    target_link_directories(harness_rnnt PRIVATE ${DALI_LIB_DIR})
-
-    target_link_libraries(harness_rnnt
-        nvinfer
-        nvinfer_plugin
-        gflags
-        glog
-        ${CUDA_LIBRARIES}
-        ${LOADGEN_LIB}
-        ${DALI_LIBRARIES}
-    )
-
-    target_include_directories(harness_rnnt
-        PUBLIC
-            ${CUDA_INCLUDE_DIRS}
-            ${LOADGEN_INCLUDE_DIR}
-            ${LWIS_INCLUDE_DIR}
-            ${DALI_INCLUDE_DIR}
-            common
-            harness_rnnt
-    )
+    #    if (${IS_SOC})
+    #        message(STATUS "Compiling RNNT harness for SM${SOC_SM}.")
+    #        target_compile_options(harness_rnnt PRIVATE $<$<COMPILE_LANGUAGE:CUDA>: -gencode arch=compute_${SOC_SM},code=sm_${SOC_SM}>)
+    #    endif()
+
+    #    target_link_directories(harness_rnnt PRIVATE ${DALI_LIB_DIR})
+
+    #    target_link_libraries(harness_rnnt
+    #        nvinfer
+    #        nvinfer_plugin
+    #        gflags
+    #        glog
+    #        ${CUDA_LIBRARIES}
+    #        ${LOADGEN_LIB}
+    #        ${DALI_LIBRARIES}
+    #    )
+
+    #    target_include_directories(harness_rnnt
+    #        PUBLIC
+    #            ${CUDA_INCLUDE_DIRS}
+    #            ${LOADGEN_INCLUDE_DIR}
+    #            ${LWIS_INCLUDE_DIR}
+    #            ${DALI_INCLUDE_DIR}
+    #            common
+    #            harness_rnnt
+    #    )


<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; make build
...
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>$ ls -la build/
total 28

total 40
drwxrwsr-x 10 katya krai 4096 May 19 16:36 .
drwxrwsr-x 15 katya krai 4096 May 19 15:35 ..
drwxr-sr-x  2 katya krai 4096 May 19 16:11 bin
lrwxrwxrwx  1 katya krai   29 May 19 16:36 data -> /datasets/nvidia_scratch/data
drwxr-sr-x  3 katya krai 4096 May 19 14:20 engines
drwxrwsr-x  5 katya krai 4096 May 19 16:11 harness
drwxrwsr-x 14 katya krai 4096 May 19 11:22 inference
drwxrwsr-x  8 katya krai 4096 May 19 16:36 logs
lrwxrwxrwx  1 katya krai   31 May 19 16:36 models -> /datasets/nvidia_scratch/models
drwxrwsr-x  6 katya krai 4096 May 19 11:36 plugins
drwxrwsr-x  9 katya krai 4096 May 19 11:22 power-dev
lrwxrwxrwx  1 katya krai   42 May 19 16:36 preprocessed_data -> /datasets/nvidia_scratch/preprocessed_data
drwxrwsr-x 10 katya krai 4096 May 19 11:23 triton-inference-server

</pre>

<a name="resnet50"></a>
# ResNet50

<a name="resnet50_offline"></a>
## Offline

<a name="resnet50_offline_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=offline&quot;
...
[2022-05-20 10:41:20,200 main.py:137 INFO] Finished building engines for resnet50 benchmark in Offline scenario.
Time taken to generate engines: 265.04155135154724 seconds

</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-Offline*.plan -exec du -hs {} \;
27M     ./build/engines/Orin/resnet50/Offline/resnet50-Offline-gpu-b256-int8.lwis_k_99_MaxP.plan
26M     ./build/engines/Orin/resnet50/Offline/resnet50-Offline-dla-b8-int8.lwis_k_99_MaxP.plan
</pre>

<a name="resnet50_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/resnet50/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 6138.84
Result is : VALID
</pre>

#### Reproduced experiment (after system reboot!)

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; 
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=offline --test_mode=PerformanceOnly&quot;
...

[2022-05-22 23:59:24,613 main.py:304 INFO] Result: result_samples_per_second: 5738.38, Result is VALID

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    resnet50: result_samples_per_second: 5738.38, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    resnet50: No accuracy results in PerformanceOnly mode.


real    11m7.798s
user    11m1.016s
sys     0m9.948s
</pre>

<a name="resnet50_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/resnet50/Offline/accuracy/accuracy.txt)

<pre>
accuracy=76.038%, good=38019, total=50000
hash=070aa1cee196b65944c4d0f7778135ab66c849bf1374eb4ccbc0aa43cddbc9ff
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; 
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=offline --test_mode=AccuracyOnly&quot;
...
accuracy=76.038%, good=38019, total=50000 TO UPDATE

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    resnet50: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    resnet50: Accuracy = 76.038, Threshold = 75.695. Accuracy test PASSED.
</pre>

<a name="resnet50_singlestream"></a>
## SingleStream - TO DO

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/resnet50/SingleStream/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/resnet50/SingleStream/config.json -A 6
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;config_ver&quot;: {
            &quot;maxq&quot;: {}
        },
        &quot;gpu_single_stream_expected_latency_ns&quot;: 2273000,
        &quot;use_graphs&quot;: false
    },
</pre>

<a name="resnet50_singlestream_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=singlestream&quot;
...
[2021-04-27 17:35:34,949 main.py:112 INFO] Finished building engines for resnet50 benchmark in SingleStream scenario.
Time taken to generate engines: 42.95994567871094 seconds

real    0m44.209s
user    0m23.192s
sys     0m4.620s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-SingleStream*.plan -exec du -hs {} \;
44M     ./build/engines/AGX_Xavier/resnet50/SingleStream/resnet50-SingleStream-gpu-b1-int8.default.plan
</pre>

<a name="resnet50_singlestream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/SingleStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
90th percentile latency (ns) : 1980582
Result is : VALID
  Min duration satisfied : Yes
  Min queries satisfied : Yes
...
QPS w/ loadgen overhead         : 509.00
QPS w/o loadgen overhead        : 512.43
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=SingleStream --test_mode=PerformanceOnly&quot;
...
================================================
MLPerf Results Summary
================================================
SUT name : LWIS_Server
Scenario : Single Stream
Mode     : Performance
90th percentile latency (ns) : 2078719
Result is : VALID
  Min duration satisfied : Yes
  Min queries satisfied : Yes

================================================
Additional Stats
================================================
QPS w/ loadgen overhead         : 485.07
QPS w/o loadgen overhead        : 488.48

Min latency (ns)                : 1980890
Max latency (ns)                : 15165301
Mean latency (ns)               : 2047146
50.00 percentile latency (ns)   : 2038621
90.00 percentile latency (ns)   : 2078719
95.00 percentile latency (ns)   : 2098912
97.00 percentile latency (ns)   : 2118465
99.00 percentile latency (ns)   : 2187844
99.90 percentile latency (ns)   : 2564821

================================================
Test Parameters Used
================================================
samples_per_query : 1
target_qps : 439.947
target_latency (ns): 0
max_async_queries : 1
min_duration (ms): 60000
max_duration (ms): 0
min_query_count : 1024
max_query_count : 0
qsl_rng_seed : 12786827339337101903
sample_index_rng_seed : 12640797754436136668
schedule_rng_seed : 3135815929913719677
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : false
performance_issue_unique : false
performance_issue_same : false
performance_issue_same_index : 0
performance_sample_count : 2048

No warnings encountered during test.

No errors encountered during test.
Finished running actual test.
Device Device:0 processed:
  29106 batches of size 1
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 0
  BatchedCudaMemcpy Calls: 29106
&&&& PASSED Default_Harness # ./build/bin/harness_default
[2021-01-29 23:25:15,883 main.py:341 INFO] Result: 90th percentile latency (ns) : 2078719 and Result is : VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-SingleStream:
    resnet50: 90th percentile latency (ns) : 2078719 and Result is : VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-SingleStream:
    resnet50: No accuracy results in PerformanceOnly mode.


real    1m16.374s
user    1m6.484s
sys     0m5.004s

</pre>

<a name="resnet50_singlestream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/SingleStream/accuracy/accuracy.txt)

<pre>
accuracy=76.064%, good=38032, total=50000
hash=7458cd3f1154670a0d063c87b38d2eba7aa8c1921f2558a46333cfef8d9b4036
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=SingleStream --test_mode=AccuracyOnly&quot;
...
accuracy=76.078%, good=38039, total=50000

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-SingleStream:
    resnet50: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-SingleStream:
    resnet50: Accuracy = 76.078, Threshold = 75.695. Accuracy test PASSED.


real    4m27.773s
user    2m9.392s
sys     0m11.332s
</pre>

<a name="resnet50_multistream"></a>
## MultiStream - TO DO

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/resnet50/MultiStream/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/resnet50/MultiStream/config.json -A 14
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;concurrent_multi_stream_samples_per_query&quot;: 96,
        &quot;config_ver&quot;: {
            &quot;maxq&quot;: {}
        },
        &quot;dla_batch_size&quot;: 15,
        &quot;dla_copy_streams&quot;: 2,
        &quot;dla_core&quot;: 0,
        &quot;dla_inference_streams&quot;: 4,
        &quot;dla_multi_stream_samples_per_query&quot;: 15,
        &quot;gpu_batch_size&quot;: 66,
        &quot;gpu_copy_streams&quot;: 2,
        &quot;gpu_inference_streams&quot;: 4,
        &quot;gpu_multi_stream_samples_per_query&quot;: 66
    },
</pre>

<a name="resnet50_multistream_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=multistream&quot;
...
[2021-04-27 20:34:39,925 main.py:112 INFO] Finished building engines for resnet50 benchmark in MultiStream scenario.
Time taken to generate engines: 118.26688885688782 seconds

real    1m59.516s
user    0m35.180s
sys     0m15.528s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-MultiStream*.plan -exec du -hs {} \;
96M     ./build/engines/AGX_Xavier/resnet50/MultiStream/resnet50-MultiStream-gpu-b66-int8.default.plan
29M     ./build/engines/AGX_Xavier/resnet50/MultiStream/resnet50-MultiStream-dla-b15-int8.default.plan
</pre>

<a name="resnet50_multistream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/MultiStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per query : 96
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=MultiStream --test_mode=PerformanceOnly&quot;
[2021-04-27 20:38:49,061 main.py:701 INFO] Detected System ID: AGX_Xavier
[2021-04-27 20:38:49,062 main.py:529 INFO] Using config files: configs/resnet50/MultiStream/config.json
[2021-04-27 20:38:49,063 __init__.py:341 INFO] Parsing config file configs/resnet50/MultiStream/config.json ...
[2021-04-27 20:38:49,063 main.py:542 INFO] Processing config "AGX_Xavier_resnet50_MultiStream"
[2021-04-27 20:38:49,064 main.py:224 INFO] Running harness for resnet50 benchmark in MultiStream scenario...
...
================================================
MLPerf Results Summary
================================================
SUT name : LWIS_Server
Scenario : MultiStream
Mode     : PerformanceOnly
Samples per query : 96
Result is : INVALID
  Performance constraints satisfied : NO
  Min duration satisfied : Yes
  Min queries satisfied : Yes
Recommendations:
 * Reduce samples per query to improve latency.

================================================
Additional Stats
================================================
Intervals between each IssueQuery:  "qps" : 20, "ms" : 50
50.00 percentile : 1
90.00 percentile : 1
95.00 percentile : 1
97.00 percentile : 1
99.00 percentile : 2
99.90 percentile : 2

Per-query latency:  "target_ns" : 50000000, "target_ms" : 50
50.00 percentile latency (ns)   : 48702492
90.00 percentile latency (ns)   : 49325203
95.00 percentile latency (ns)   : 49511578
97.00 percentile latency (ns)   : 49645776
99.00 percentile latency (ns)   : 49978295
99.90 percentile latency (ns)   : 50705066

Per-sample latency:
Min latency (ns)                : 45861628
Max latency (ns)                : 182167011
Mean latency (ns)               : 48390461
50.00 percentile latency (ns)   : 48353859
90.00 percentile latency (ns)   : 49233149
95.00 percentile latency (ns)   : 49415884
97.00 percentile latency (ns)   : 49546249
99.00 percentile latency (ns)   : 49851112
99.90 percentile latency (ns)   : 50605105

================================================
Test Parameters Used
================================================
samples_per_query : 96
target_qps : 20
target_latency (ns): 50000000
max_async_queries : 1
min_duration (ms): 600000
max_duration (ms): 0
min_query_count : 270336
max_query_count : 0
qsl_rng_seed : 7322528924094909334
sample_index_rng_seed : 1570999273408051088
schedule_rng_seed : 3507442325620259414
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : 0
performance_issue_unique : 0
performance_issue_same : 0
performance_issue_same_index : 0
performance_sample_count : 2048

No warnings encountered during test.

No errors encountered during test.
...
[2021-04-28 10:17:24,243 main.py:280 INFO] Result: requested_multi_stream_samples_per_query: 96, Result is INVALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-MultiStream:
    resnet50: requested_multi_stream_samples_per_query: 96, Result is INVALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-MultiStream:
    resnet50: No accuracy results in PerformanceOnly mode.


real    228m2.179s
user    12m59.700s
sys     3m27.784s
</pre>

<a name="resnet50_multistream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/MultiStream/accuracy/accuracy.txt)

<pre>
accuracy=76.008%, good=38004, total=50000
hash=12988c6c56d7af58b5e816d9b9d64b20f5ae80497e355dc51653ed2433b583bb
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=MultiStream --test_mode=AccuracyOnly&quot;
...
accuracy=76.020%, good=38010, total=50000

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-MultiStream:
    resnet50: requested_multi_stream_samples_per_query: 96, Result validity unknown


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-MultiStream:
    resnet50: Accuracy = 76.020, Threshold = 75.695. Accuracy test PASSED.


real    1m2.343s
user    0m24.100s
sys     0m7.468s
</pre>

<a name="bert"></a>
# BERT

<a name="bert_offline"></a>
## Offline

<a name="bert_offline_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; make generate_engines RUN_ARGS=&quot;--benchmarks=bert --scenarios=offline&quot;
...
Time taken to generate engines: 4379.266929149628 seconds

real    73m3.881s
user    0m32.668s
sys     16m22.002s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name bert-Offline*.plan -exec du -hs {} \;
355M    ./build/engines/Orin/bert/Offline/bert-Offline-gpu-int8_S_384_B_256_P_1_vs_il.custom_k_99_MaxP.plan
</pre>

<a name="bert_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/bert-99/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 476.344
Result is : VALID
</pre>

#### Reproduced experiment (after system reboot!)

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; 
make run_harness RUN_ARGS=&quot;--benchmarks=bert --scenarios=offline --test_mode=PerformanceOnly&quot;
...

[2022-05-23 02:00:20,632 main.py:304 INFO] Result: result_samples_per_second: 482.61, Result is VALID

======================= Perf harness results: =======================

Orin_TRT-custom_k_99_MaxP-Offline:
    bert: result_samples_per_second: 482.61, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-custom_k_99_MaxP-Offline:
    bert: No accuracy results in PerformanceOnly mode.


real    11m18.494s
user    0m9.500s
sys     0m2.521s
</pre>

<a name="bert_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/bert-99/Offline/accuracy/accuracy.txt)

<pre>
{"exact_match": 82.687, "f1": 90.104}
hash=f56718adf61873042f22ce72c152f015b8ac1057a658676ba39c505c2b6f22cf
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; 
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=offline --test_mode=AccuracyOnly&quot;
...
======================= Accuracy results: =======================

Orin_TRT-custom_k_99_MaxP-Offline:
    bert: Accuracy = 90.104, Threshold = 89.965. Accuracy test PASSED.


real    1m54.490s
user    1m17.205s
sys     0m1.996s
</pre>

<a name="ssd-resnet34"></a>
# SSD-ResNet34 - TO DO

<a name="ssd-resnet34_offline"></a>
## Offline

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/ssd-resnet34/Offline/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/ssd-resnet34/Offline/config.json -A 16
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;concurrent_offline_expected_qps&quot;: 56,
        &quot;config_ver&quot;: {
            &quot;maxq&quot;: {
                &quot;concurrent_offline_expected_qps&quot;: 42
            }
        },
        &quot;dla_batch_size&quot;: 1,
        &quot;dla_copy_streams&quot;: 1,
        &quot;dla_core&quot;: 0,
        &quot;dla_inference_streams&quot;: 1,
        &quot;dla_offline_expected_qps&quot;: 10,
        &quot;gpu_batch_size&quot;: 2,
        &quot;gpu_copy_streams&quot;: 4,
        &quot;gpu_inference_streams&quot;: 1,
        &quot;gpu_offline_expected_qps&quot;: 35.1243
    },
</pre>

<a name="ssd-resnet34_offline_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=offline&quot;
...
[2021-04-28 21:32:57,015 main.py:112 INFO] Finished building engines for ssd-resnet34 benchmark in Offline scenario.
Time taken to generate engines: 381.31431436538696 seconds

real    6m23.144s
user    2m16.348s
sys     0m46.172s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-Offline*.plan -exec du -hs {} \;
147M    ./build/engines/AGX_Xavier/ssd-resnet34/Offline/ssd-resnet34-Offline-gpu-b2-int8.default.plan
23M     ./build/engines/AGX_Xavier/ssd-resnet34/Offline/ssd-resnet34-Offline-dla-b1-int8.default.plan
</pre>

<a name="ssd-resnet34_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 56.6721
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=Offline --test_mode=PerformanceOnly&quot;
...
================================================
MLPerf Results Summary
================================================
SUT name : LWIS_Server
Scenario : Offline
Mode     : PerformanceOnly
Samples per second: 51.1607
Result is : VALID
  Min duration satisfied : Yes
  Min queries satisfied : Yes

================================================
Additional Stats
================================================
Min latency (ns)                : 195940306
Max latency (ns)                : 722430107208
Mean latency (ns)               : 361257062980
50.00 percentile latency (ns)   : 361271790879
90.00 percentile latency (ns)   : 650292988467
95.00 percentile latency (ns)   : 686348820151
97.00 percentile latency (ns)   : 700764011699
99.00 percentile latency (ns)   : 715213570646
99.90 percentile latency (ns)   : 721724962037

================================================
Test Parameters Used
================================================
samples_per_query : 36960
target_qps : 56
target_latency (ns): 0
max_async_queries : 1
min_duration (ms): 600000
max_duration (ms): 0
min_query_count : 1
max_query_count : 0
qsl_rng_seed : 7322528924094909334
sample_index_rng_seed : 1570999273408051088
schedule_rng_seed : 3507442325620259414
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : 0
performance_issue_unique : 0
performance_issue_same : 0
performance_issue_same_index : 0
performance_sample_count : 64

No warnings encountered during test.

No errors encountered during test.
...
[2021-04-28 21:49:22,497 main.py:280 INFO] Result: result_samples_per_second: 51.1607, Result is VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-Offline:
    ssd-resnet34: result_samples_per_second: 51.1607, Result is VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-Offline:
    ssd-resnet34: No accuracy results in PerformanceOnly mode.


real    12m18.608s
user    12m4.692s
sys     0m8.680s
</pre>

<a name="ssd-resnet34_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/Offline/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.379
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.237
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.180
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.429
mAP=20.052%
hash=74f272c16708cc31a2c487a16086c6d6a03b121cf34156f94b036c4fad0ad7ac
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=Offline --test_mode=AccuracyOnly&quot;
...
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.380
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.427
mAP=20.031%

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-Offline:
    ssd-resnet34: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-Offline:
    ssd-resnet34: Accuracy = 20.031, Threshold = 19.800. Accuracy test PASSED.


real    10m50.156s
user    6m3.140s
sys     0m23.652s
</pre>

<a name="ssd-resnet34_singlestream"></a>
## SingleStream

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/ssd-resnet34/SingleStream/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/ssd-resnet34/SingleStream/config.json -A 11
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;gpu_batch_size&quot;: 1,
        &quot;gpu_copy_streams&quot;: 1,
        &quot;gpu_inference_streams&quot;: 1,
        &quot;gpu_single_stream_expected_latency_ns&quot;: 29478000,
        &quot;input_dtype&quot;: &quot;int8&quot;,
        &quot;input_format&quot;: &quot;linear&quot;,
        &quot;map_path&quot;: &quot;data_maps/coco/val_map.txt&quot;,
        &quot;precision&quot;: &quot;int8&quot;,
        &quot;tensor_path&quot;: &quot;${PREPROCESSED_DATA_DIR}/coco/val2017/SSDResNet34/int8_linear&quot;,
        &quot;use_graphs&quot;: false
    },
</pre>

<a name="ssd-resnet34_singlestream_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=singlestream&quot;
...
[2021-01-29 17:45:05,185 main.py:153 INFO] Finished building engines for ssd-resnet34 benchmark in SingleStream scenario.
Time taken to generate engines: 63.35827445983887 seconds

real    1m5.118s
user    0m15.400s
sys     0m5.076s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-SingleStream*.plan -exec du -hs {} \;
37M     ./build/engines/AGX_Xavier/ssd-resnet34/SingleStream/ssd-resnet34-SingleStream-gpu-b1-int8.default.plan
</pre>

<a name="ssd-resnet34_singlestream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/SingleStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
90th percentile latency (ns) : 28531845
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=SingleStream --test_mode=PerformanceOnly&quot;
...
================================================
MLPerf Results Summary
================================================
SUT name : LWIS_Server
Scenario : Single Stream
Mode     : Performance
90th percentile latency (ns) : 28554901
Result is : VALID
  Min duration satisfied : Yes
  Min queries satisfied : Yes

================================================
Additional Stats
================================================
QPS w/ loadgen overhead   : 35.17
QPS w/o loadgen overhead  : 35.23

Min latency (ns)    : 28109905
Max latency (ns)    : 32556991
Mean latency (ns)   : 28385289
50.00 percentile latency (ns)   : 28361924
90.00 percentile latency (ns)   : 28554901
95.00 percentile latency (ns)   : 28625392
97.00 percentile latency (ns)   : 28688816
99.00 percentile latency (ns)   : 28884610
99.90 percentile latency (ns)   : 29821107

================================================
Test Parameters Used
================================================
samples_per_query : 1
target_qps : 33.9236
target_latency (ns): 0
max_async_queries : 1
min_duration (ms): 60000
max_duration (ms): 0
min_query_count : 1024
max_query_count : 0
qsl_rng_seed : 12786827339337101903
sample_index_rng_seed : 12640797754436136668
schedule_rng_seed : 3135815929913719677
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : false
performance_issue_unique : false
performance_issue_same : false
performance_issue_same_index : 0
performance_sample_count : 64

No warnings encountered during test.

No errors encountered during test.
Finished running actual test.
Device Device:0 processed:
  2111 batches of size 1
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 0
  BatchedCudaMemcpy Calls: 2111
&&&& PASSED Default_Harness # ./build/bin/harness_default
[2021-01-29 22:39:22,362 main.py:341 INFO] Result: 90th percentile latency (ns) : 28554901 and Result is : VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-SingleStream:
    ssd-resnet34: 90th percentile latency (ns) : 28554901 and Result is : VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-SingleStream:
    ssd-resnet34: No accuracy results in PerformanceOnly mode.


real    1m9.437s
user    1m4.720s
sys     0m3.964s
</pre>

<a name="ssd-resnet34_singlestream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/SingleStream/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.111%
hash=526aac286ebb67218a3528397b4aecbff9269cbe01307069569345d9c3fbb445
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=SingleStream --test_mode=AccuracyOnly&quot;
...
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.111%

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-SingleStream:
    ssd-resnet34: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-SingleStream:
    ssd-resnet34: Accuracy = 20.111, Threshold = 19.800. Accuracy test PASSED.


real    12m55.238s
user    8m38.988s
sys     0m16.408s
</pre>

<a name="ssd-resnet34_multistream"></a>
## MultiStream

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/ssd-resnet34/MultiStream/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/ssd-resnet34/MultiStream/config.json -A 9
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;gpu_batch_size&quot;: 2,
        &quot;gpu_multi_stream_samples_per_query&quot;: 2,
        &quot;input_dtype&quot;: &quot;int8&quot;,
        &quot;input_format&quot;: &quot;linear&quot;,
        &quot;map_path&quot;: &quot;data_maps/coco/val_map.txt&quot;,
        &quot;precision&quot;: &quot;int8&quot;,
        &quot;tensor_path&quot;: &quot;${PREPROCESSED_DATA_DIR}/coco/val2017/SSDResNet34/int8_linear&quot;,
        &quot;use_graphs&quot;: false
    },
</pre>

<a name="ssd-resnet34_multistream_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=multistream&quot;
...
[2021-01-29 07:33:48,592 main.py:153 INFO] Finished building engines for ssd-resnet34 benchmark in MultiStream scenario.
Time taken to generate engines: 263.8264467716217 seconds

real    4m25.496s
user    0m32.000s
sys     0m35.784s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-MultiStream*.plan -exec du -hs {} \;
148M    ./build/engines/AGX_Xavier/ssd-resnet34/MultiStream/ssd-resnet34-MultiStream-gpu-b2-int8.default.plan
</pre>

<a name="ssd-resnet34_multistream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/MultiStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per query : 2
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=MultiStream --test_mode=PerformanceOnly&quot;
...
================================================
MLPerf Results Summary
================================================
SUT name : LWIS_Server
Scenario : Multi Stream
Mode     : Performance
Samples per query : 2
Result is : VALID
  Performance constraints satisfied : Yes
  Min duration satisfied : Yes
  Min queries satisfied : Yes

================================================
Additional Stats
================================================
Intervals between each IssueQuery:  "qps" : 15, "ms" : 66.6667
50.00 percentile : 1
90.00 percentile : 1
95.00 percentile : 1
97.00 percentile : 1
99.00 percentile : 1
99.90 percentile : 1

Per-query latency:  "target_ns" : 66666666, "target_ms" : 66.6667
50.00 percentile latency (ns)   : 55624812
90.00 percentile latency (ns)   : 55913099
95.00 percentile latency (ns)   : 56006924
97.00 percentile latency (ns)   : 56067621
99.00 percentile latency (ns)   : 56189283
99.90 percentile latency (ns)   : 56401684

Per-sample latency:
Min latency (ns)                : 55180175
Max latency (ns)                : 61310689
Mean latency (ns)               : 55646311
50.00 percentile latency (ns)   : 55624812
90.00 percentile latency (ns)   : 55913099
95.00 percentile latency (ns)   : 56006924
97.00 percentile latency (ns)   : 56067621
99.00 percentile latency (ns)   : 56189283
99.90 percentile latency (ns)   : 56401684

================================================
Test Parameters Used
================================================
samples_per_query : 2
target_qps : 15
target_latency (ns): 66666666
max_async_queries : 1
min_duration (ms): 60000
max_duration (ms): 0
min_query_count : 270336
max_query_count : 0
qsl_rng_seed : 12786827339337101903
sample_index_rng_seed : 12640797754436136668
schedule_rng_seed : 3135815929913719677
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : false
performance_issue_unique : false
performance_issue_same : false
performance_issue_same_index : 0
performance_sample_count : 64

No warnings encountered during test.

No errors encountered during test.
Finished running actual test.
Equivalent QPS computed by samples_per_query*target_qps : 30
Device Device:0 processed:
  270336 batches of size 2
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 0
  BatchedCudaMemcpy Calls: 270336
&&&& PASSED Default_Harness # ./build/bin/harness_default
[2021-01-29 13:12:12,503 main.py:341 INFO] Result: Samples per query : 2 and Result is : VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-MultiStream:
    ssd-resnet34: Samples per query : 2 and Result is : VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-MultiStream:
    ssd-resnet34: No accuracy results in PerformanceOnly mode.


real    300m37.676s
user    5m11.760s
sys     0m27.856s
</pre>

<a name="ssd-resnet34_multistream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/MultiStream/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.111%
hash=859242388c9a94513b189eb58a55bd11ad1d2f7d094880dfb72157a7ac5e45fd
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=MultiStream --test_mode=AccuracyOnly&quot;
...
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.111%

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-MultiStream:
    ssd-resnet34: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-MultiStream:
    ssd-resnet34: Accuracy = 20.111, Threshold = 19.800. Accuracy test PASSED.


real    11m44.285s
user    4m22.432s
sys     0m17.144s
</pre>